# Sequelize

> The document can find [here](https://sequelize.org/master/manual/getting-started.html)

## 1. Setup

### 1.1. Make connection

In [ ]:
const {Sequelize, DataTypes, QueryTypes, Op} = require('sequelize');

const sequelize = new Sequelize('study_node', 'root', 'root', {
    host: 'localhost',
    port: 3306,
    dialect: 'mysql',
    pool: {
        max: 5,
        min: 0,
        idle: 10000
    }
});

### 1.2. Create tables

- Functions

In [ ]:
const fs = require('fs');
const path = require('path');

async function createTables() {
    /** @type {any}*/
    let sqls = await fs.promises.readFile(path.join(process.cwd(), 'tables.sql'), 'utf-8');
    console.log(`* read sql from file:`);
    console.log(sqls);
    
    sqls = sqls.split(';').map(sql => sql.trim()).filter(sql => !!sql);
    
    const options = {type: QueryTypes.RAW};
    
    return await sequelize.transaction(async tx => {
        const tasks = [];
        for (let sql of sqls) {
            tasks.push(await sequelize.query(sql, options));
        }
        return await tasks;
    });
}

- Test

In [ ]:
{
    await createTables();
    console.log(`* all tables created`);
}

### 1.3. Truncate tabel

- Functions

In [ ]:
async function truncateTables() {
    const options = {type: QueryTypes.RAW};
    return sequelize.transaction(async tx => {
        return [
            await sequelize.query('SET FOREIGN_KEY_CHECKS = 0;', options),
            await sequelize.query('TRUNCATE TABLE user;', options),
            await sequelize.query('TRUNCATE TABLE project;', options),
            await sequelize.query('SET FOREIGN_KEY_CHECKS = 1;', options)
        ];
    });
}

- Test

In [ ]:
{
    await truncateTables();
    console.log(`* tables truncated`);
}

## 2. Create entity models

### 2.1. Define models

In [ ]:
const DEFAULT_OPTS = {
    freezeTableName: true,
    timestamps: false
};

/** 
 * @type {any}
 * Define User entity for table 'user'
 */
const User = sequelize.define('user', {
    id: {type: DataTypes.INTEGER, field: 'id', primaryKey: true, autoIncrement: true},
    name: {type: DataTypes.STRING, field: 'name'},
    gender: {type: DataTypes.STRING, field: 'gender'},
    birthday: {type: DataTypes.DATE, field: 'birthday', allowNull: true},
    phone: {type: DataTypes.STRING, field: 'phone'},
    projectId: {type: DataTypes.INTEGER, field: 'project_id'}
}, DEFAULT_OPTS);

/** 
 * @type {any}
 * Define Project entity for table 'project'
 */
const Project = sequelize.define('project', {
    id: {type: DataTypes.INTEGER, field: 'id', primaryKey: true, autoIncrement: true},
    name: {type: DataTypes.STRING, field: 'name'},
    type: {type: DataTypes.STRING, field: 'type'}
}, DEFAULT_OPTS);

### 2.2. Define Relationship

In [ ]:
// User=>Project: Many to one
User.belongsTo(Project, {as: 'project', foreignKey: 'project_id', targetKey: 'id'});

// Project=>User: One to many
Project.hasMany(User, {as: 'users', foreignKey: 'project_id'});

## 3. Query

### 3.1. Create data

- Function

In [ ]:
const moment = require('moment');

async function createProject() {
    return sequelize.transaction(async tx => {
        return await Project.create({
            name: 'ROOMIS',
            type: 'DEV'
        });
    });
}

async function createUsers(project) {
    return sequelize.transaction(async tx => {
        return await Promise.all([
            User.create({
                name: 'Alvin',
                gender: 'M',
                birthday: moment.utc('1981-03-17'),
                phone: '13991320110',
                projectId: project.id
            }),
            User.create({
                name: 'Emma',
                gender: 'F',
                birthday: moment.utc('1985-03-29'),
                phone: '13991320112',
                projectId: project.id
            })
        ]);
    });
}

- Test

In [ ]:
{
    const project = await createProject();
    console.log('* project created');
    
    await createUsers(project);
    console.log('* users created');
}

### 3.2. Query user

#### 3.2.1. Query user by name

- Function

In [ ]:
async function findUserByName(name) {
    return await User.findAll({
        where: {
            name: name
        }
    });
}

- Test

In [ ]:
{
    const users = await findUserByName('Alvin');
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 3.2.2. Find user with some attribute

- Function

In [ ]:
async function findUserNameAndGenderByName(name) {
    return await User.findAll({
        attributes: ['name', 'gender'],
        where: {
            name: name
        }
    });
}

- Test

In [ ]:
{
    const users = await findUserNameAndGenderByName('Emma');
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 3.2.3. Query user with sql function

- Function

In [ ]:
async function findUserNameLengthByName(name) {
    return await User.findAll({
        attributes: ['name', [Sequelize.fn('length', Sequelize.col('name')), 'length']],
        where: {
            name: name
        }
    });
}

- Test

In [ ]:
{
    const users = await findUserNameLengthByName('Alvin');
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 3.2.4. Query user use like operator

- Function

In [ ]:
async function findUserByNameLike(name) {
    return await User.findAll({
        where: {
            name: {
                [Op.like]: `${name}%`
            }
        }
    });
}

- Test

In [ ]:
{
    const users = await findUserByNameLike('A');
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 3.2.5. Query by more conditions

- Function

In [ ]:
async function findUserByNameGenderAndBirthYear(name, gender, birthYear) {
    const beginDate = moment().utc().year(birthYear).month(1).startOf('month');
    const endDate = moment().utc().year(birthYear).month(12).endOf('month');

    return await User.findAll({
        where: {
            [Op.and]: {
                name: name,
                gender: gender
            },
            birthday: {
                [Op.or]: {
                    [Op.gte]: beginDate,
                    [Op.lte]: endDate
                }
            }
        }
    });
}

- Test

In [ ]:
{
    const users = await findUserByNameGenderAndBirthYear('Alvin', 'M', 1981);
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 3.2.5 Query pagination

- Functions

In [ ]:
async function findUserByNameAndPage(name, pagination) {
    const offset = (pagination.page - 1) * pagination.perpage;
    const limit = pagination.perpage;
          
    return await User.findAll({
        where: {
            name: {
                [Op.like]: `${name}%`
            }
        },
        offset, limit
    });
}

- Test

In [ ]:
{
    const users = await findUserByNameAndPage('A', {
        page: 1,
        perpage: 10
    });
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 2.3.6. Count query

- Function

In [ ]:
async function countUserByGender(gender) {
    return await User.count({
        where: {
            gender: gender
        }
    });
}

- Test

In [ ]:
{
    const count = await countUserByGender('M');
    console.log(`* there are ${count} user by gender 'M'`);
}

#### 2.3.7. Query by order

- Function

In [ ]:
async function queryUserOrderByBirthday() {
    return await User.findAll({
        order: [
            ['birthday', 'DESC']
        ]
    });
}

- Test

In [ ]:
{
    const users = await queryUserOrderByBirthday();
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.id}: ${JSON.stringify(user.toJSON())}`);
    }
}

#### 2.3.8. Grouping

- Function

In [ ]:
async function queryMaxBirthdayByGender() {
    return await User.findAll({
        attributes: ['gender', [Sequelize.fn('max', Sequelize.col('birthday')), 'maxBirthday']],
        group: 'gender'
    });
}

- Test

In [ ]:
{
    const users = await queryMaxBirthdayByGender();
    console.log(`* there are ${users.length} users found.`);
    
    for (const user of users) {
        console.log(`\t user ${user.gender}: ${moment(user.get('maxBirthday')).utc().format()}`);
    }
}

### 3.3. Relationship

#### 3.3.1. Query by relationship

- Function

In [ ]:
async function findProjectWithUsers(name) {
    const projects = await Project.findAll({
        where: {
            name: name
        }
    });
    
    const result = [];
    for (let pro of projects) {
        result.push({project: pro, users: await pro.getUsers()});
    }
    return result;
}

- Test

In [ ]:
{
    const groups = await findProjectWithUsers('ROOMIS');
    for (let group of groups) {
        console.log(`* project ${JSON.stringify(group.project.toJSON())}: `);
        
        for (let user of group.users) {
            console.log(`\t user ${JSON.stringify(user.toJSON())}: `);
        }
    }
}

#### 3.3.2. Query by join

- Function

In [ ]:
async function findProjectJoinUsers(name) {
    return await Project.findAll({
        where: {
            name: name
        },
        include: [
             {model: User, as: 'users', required: true}   // required: [true: inner join, false: left outer join]
        ]
    });
}

- Test

In [ ]:
{
    const projects = await findProjectJoinUsers('ROOMIS');
    for (let pro of projects) {
        console.log(`* project ${JSON.stringify({id: pro.id, name: pro.name, type: pro.type})}: `);
        
        for (let user of pro.users) {  // do not use 'pro.getUsers()', it will generate new sql query
            console.log(`\t user ${JSON.stringify(user.toJSON())}: `);
        }
    }
}

### 3.4. Update

#### 3.4.1. Update User by model

- Function

In [ ]:
async function updateUserModelByName(name, gender, birthday, phone) {
    return await sequelize.transaction(async tx => {
        const users = await findUserByName(name);
        if (users.length === 0) {
            return;
        }
        const user = users[0];
        user.set('gender', gender);
        user.set('birthday', birthday);
        user.set('phone', phone);
        return await user.save();
    });
}

- Test

In [ ]:
{
    await updateUserModelByName('Alvin', 'M', moment.utc('1981-03-18'), '13999991919');
    const user = (await findUserByName('Alvin'))[0];
    console.log(`* after update, user is: ${JSON.stringify(user.toJSON())}`);
}

#### 3.4.2. Update user by condition

- Function

In [ ]:
async function updateUserByNameCondition(name, gender, birthday, phone) {
    return await sequelize.transaction(async tx => {
        return await User.update({
            gender: gender,
            birthday: birthday,
            phone: phone
        }, {
            where: {
                name: name
            }
        });
    });
}

- Test

In [ ]:
{
    await updateUserByNameCondition('Alvin', 'M', moment.utc('1981-03-17'), '13990909099');
    const user = (await findUserByName('Alvin'))[0];
    console.log(`* after update, user is: ${JSON.stringify(user.toJSON())}`);
}

### 3.5. Delete

#### 3.5.1. Delete user by model

- Function

In [ ]:
async function deleteUserModelByName(name) {
    return await sequelize.transaction(async tx => {
        const users = await findUserByName(name);
        if (users.length === 0) {
            return;
        }
        return await users[0].destroy();
    });
}

- Test

In [ ]:
{
    await deleteUserModelByName('Alvin');
    const users = await findUserByName('Alvin');
    console.log(`* after update, ${users.length} users name by 'Alvin'`);
}

#### 3.5.2. Delete user by condition

- Function

In [ ]:
async function deleteUserByNameCondition(name) {
    return await sequelize.transaction(async tx => {
        return User.destroy({
            where: {
                name: name
            }
        });
    });
}

- Test

In [ ]:
{
    await deleteUserByNameCondition('Emma');
    const users = await findUserByName('Emma');
    console.log(`* after update, ${users.length} users name by 'Emma'`);
}